In [0]:
from IPython.display import clear_output
!pip install --upgrade tensorflow-gpu
!pip install --upgrade tqdm
clear_output()

In [0]:
#Downloads and extract Dataset to local, wait for download, i dont want to put a progress bar here sorry
#You can run this on google colab for get faster downloads speeds
import os
import tarfile
import requests
from tqdm import tqdm

folder_path = "./Datasets"

wiki_url = "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar"
wiki_path = folder_path + '/wiki.tar.gz'

imdb_url = "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar"
imdb_path = folder_path + '/imdb.tar.gz'

if(not os.path.exists(folder_path)):
  os.mkdir(folder_path)

if(not(os.path.exists(wiki_path) and os.path.isfile(wiki_path))):
  resp = requests.get(wiki_url, stream=True)

  total_size = int(resp.headers.get('content-length', 0))
  block_size = 16384
  t=tqdm(total=total_size, unit='iB', unit_scale=True)
  
  with open(wiki_path, "wb") as f:
    for data in resp.iter_content(block_size):
      t.update(len(data))
      f.write(data)
    t.close()
    f.close()

    if total_size != 0 and t.n != total_size:
      print("Wiki Faces Download Error")

  tar = tarfile.open(wiki_path, "r")
  tar.extractall(folder_path)
  tar.close()

  os.remove("./Datasets/wiki_crop/wiki.mat")



if(not(os.path.exists(imdb_path) and os.path.isfile(imdb_path))):
  resp = requests.get(imdb_url, stream=True)

  total_size = int(resp.headers.get('content-length', 0))
  block_size = 16384
  t=tqdm(total=total_size, unit='iB', unit_scale=True)
  
  with open(imdb_path, "wb") as f:
    for data in resp.iter_content(block_size):
      t.update(len(data))
      f.write(data)
    t.close()
    f.close()

    if total_size != 0 and t.n != total_size:
      print("Imdb Faces Download Error")

  tar = tarfile.open(imdb_path, "r")
  tar.extractall(folder_path)
  tar.close()

  os.remove("./Datasets/imdb_crop/imdb.mat")

In [0]:
#Imports
import tensorflow as tf
import numpy as np
import pandas as pd

In [0]:
root = "./Datasets"
folders = os.listdir(root)

if(not ("dataset.csv" in folders)):
  folders.remove("imdb.tar.gz")
  folders.remove("wiki.tar.gz")

  data = list()

  for folder in folders:
    subfolders = os.listdir("{root}/{folder}".format(root=root, folder=folder))
    for subfolder in subfolders:
      files = os.listdir("{root}/{folder}/{subfolder}".format(root=root, folder=folder, subfolder=subfolder))
      for file in files:
        path = "{root}/{folder}/{subfolder}/{file}".format(root=root, folder=folder, subfolder=subfolder, file=file)

        try:
          file_data = file[:-4]
          file_data = file_data.split("_")

          if(folder == "wiki_crop"):
            id = file_data[0]
            born = int(file_data[1].split("-")[0])
          else:
            id = file_data[0][2:]
            born = int(file_data[2].split("-")[0])

          taken = int(file_data[-1])
          age = taken - born

          data.append([id, age, path])
        except:
          print("error")
          

  df = pd.DataFrame(data=data, columns=["personId", "age", "url"])
  df.to_csv(root + "/dataset.csv")
  df.head()

In [0]:
#Load csv
faces_df = pd.read_csv("./Datasets/dataset.csv")